# 將原始標註資料轉換為YOLOv5格式

In [2]:
import os
import pandas as pd

In [4]:
# Use the public_training_data.csv for YOLOv5 training
train_df = pd.read_csv('./public_training_data.csv')
#train_df.columns

## 將超出圖片範圍的座標點clip到邊界，並且取座標的左上角與右下角當作bounding box

In [5]:
# image_size: 1232x1028
# x_center, y_center, width, height
# Clip negative bbox position to 0
def clean_bbox(df, image_width, image_height):
    df.loc[df['top left x'] < 0, 'top left x'] = 0.0
    df.loc[df['top left y'] < 0, 'top left y'] = 0.0
    df.loc[df['top right x'] < 0, 'top right x'] = 0.0
    df.loc[df['top right y'] < 0, 'top right y'] = 0.0
    df.loc[df['bottom left x'] < 0, 'bottom left x'] = 0.0
    df.loc[df['bottom left y'] < 0, 'bottom left y'] = 0.0
    df.loc[df['bottom right x'] < 0, 'bottom right x'] = 0.0
    df.loc[df['bottom right y'] < 0, 'bottom right y'] = 0.0
    
    df.loc[df['top left x'] > image_width, 'top left x'] = image_width
    df.loc[df['top left y'] > image_height, 'top left y'] = image_height
    df.loc[df['top right x'] > image_width, 'top right x'] = image_width
    df.loc[df['top right y'] > image_height, 'top right y'] = image_height
    df.loc[df['bottom left x'] > image_width, 'bottom left x'] = image_width
    df.loc[df['bottom left y'] > image_height, 'bottom left y'] = image_height
    df.loc[df['bottom right x'] > image_width, 'bottom right x'] = image_width
    df.loc[df['bottom right y'] > image_height, 'bottom right y'] = image_height
    return df

image_width = 1232
image_height = 1028

train_df['yolo_label'] = '0'
train_df = clean_bbox(train_df, image_width, image_height)
train_df['x_min'] = train_df.loc[:, ['top left x', 'bottom left x', 'top right x', 'bottom right x']].min(axis=1)
train_df['y_min'] = train_df.loc[:, ['top left y', 'bottom left y', 'top right y', 'bottom right y']].min(axis=1)
train_df['x_max'] = train_df.loc[:, ['top left x', 'bottom left x', 'top right x', 'bottom right x']].max(axis=1)
train_df['y_max'] = train_df.loc[:, ['top left y', 'bottom left y', 'top right y', 'bottom right y']].max(axis=1)
train_df['width'] = train_df['x_max'] - train_df['x_min']
train_df['height'] = train_df['y_max'] - train_df['y_min']
train_df['x_center'] = train_df['x_min'] + train_df['width']/2
train_df['y_center'] = train_df['y_min'] + train_df['height']/2

# Normalize the bbox by image size
train_df['width'] = train_df['width'] / image_width
train_df['height'] = train_df['height'] / image_height
train_df['x_center'] = train_df['x_center'] / image_width
train_df['y_center'] = train_df['y_center'] / image_height
train_df.head()

,filename,label,top right x,top right y,bottom right x,bottom right y,bottom left x,bottom left y,top left x,top left y,yolo_label,x_min,y_min,x_max,y_max,width,height,x_center,y_center
0,oR72wdOuzdHVh3sMMmC4gMG2aG5jeub,3LN996012F,706.84430,411.87338,707.51070,455.82960,371.28748,462.13885,370.62106,418.18262,0,370.62106,411.87338,707.51070,462.13885,0.273449,0.048896,0.437553,0.425103
1,7VVWhb6tKde=x=w5x6_p=lfcTlGTBa,2FV271041L,767.03550,426.20898,767.60080,488.00024,428.13678,491.84418,427.57147,430.05290,0,427.57147,426.20898,767.60080,491.84418,0.275998,0.063847,0.485054,0.446524
2,VweTPTTYjERnFWFr3R7=YFKrj9NBT1Lg,2FV413012A,840.83276,495.59230,841.15540,551.61390,421.96686,554.60376,421.64413,498.58215,0,421.64413,495.59230,841.15540,554.60376,0.340512,0.057404,0.512500,0.510796
3,XOIg4ZPERXLzBmsdMd4XNHYH1r=Urbr,3LP563013A,871.62260,522.91460,872.52136,593.15930,352.70407,601.39160,351.80527,531.14690,0,351.80527,522.91460,872.52136,601.39160,0.422659,0.076339,0.496886,0.546842
4,jm85agd8RyzlDBl6jjnpypYIuVw7BTvb,3LR436033F,837.74120,424.97200,838.30910,474.88922,469.82086,480.07870,469.25296,430.16144,0,469.25296,424.97200,838.30910,480.07870,0.299559,0.053606,0.530666,0.440200


In [6]:
# YOLOv5格式: filename label_id x_center y_center width height
train_df.loc[:, ['filename', 'yolo_label', 'x_center', 'y_center', 'width', 'height']].head()

,filename,yolo_label,x_center,y_center,width,height
0,oR72wdOuzdHVh3sMMmC4gMG2aG5jeub,0,0.437553,0.425103,0.273449,0.048896
1,7VVWhb6tKde=x=w5x6_p=lfcTlGTBa,0,0.485054,0.446524,0.275998,0.063847
2,VweTPTTYjERnFWFr3R7=YFKrj9NBT1Lg,0,0.512500,0.510796,0.340512,0.057404
3,XOIg4ZPERXLzBmsdMd4XNHYH1r=Urbr,0,0.496886,0.546842,0.422659,0.076339
4,jm85agd8RyzlDBl6jjnpypYIuVw7BTvb,0,0.530666,0.440200,0.299559,0.053606


## 設定要匯出的路徑，並輸出標注檔。

In [71]:
from tqdm import tqdm
output_dir = './public_yolov5_data/train_split/labels'

train_df['yolo_label'] = train_df['yolo_label'].astype(str)
train_df['x_center'] = train_df['x_center'].astype(str)
train_df['y_center'] = train_df['y_center'].astype(str)
train_df['width'] = train_df['width'].astype(str)
train_df['height'] = train_df['height'].astype(str)

for i in tqdm(train_df.index):
    output_path = os.path.join(output_dir, f"{train_df.loc[i, 'filename']}.txt")
    with open(output_path, 'w') as f:
        bbox_str = ' '.join(train_df.loc[i, ['yolo_label', 'x_center', 'y_center', 'width', 'height']].to_list())
        f.write(bbox_str)


100%|██████████| 12067/12067 [00:09<00:00, 1315.80it/s]


# 將YOLOv5資料集切分成training與validation

In [87]:
import shutil
# 設定資料集路徑
src_dir = './public_yolov5_data/train_split'
image_src = os.path.join(src_dir, 'images')
label_src = os.path.join(src_dir, 'labels')

dst_dir = './public_yolov5_data/val_split'
image_dst = os.path.join(dst_dir, 'images')
label_dst = os.path.join(dst_dir, 'labels')

all_images = sorted(os.listdir(image_src))
all_labels = sorted(os.listdir(label_src))
print(len(all_images))

12067


## 設定validation dataset的比例並且計算數量

In [88]:
val_ratio = 0.2
val_size = int(len(all_images) * val_ratio)
print(val_size)

2413


## 將val_ratio比例的資料搬移到dst_dir

In [89]:
for img, lab in tqdm(zip(all_images[-val_size:], all_labels[-val_size:])):
    shutil.move(os.path.join(image_src, img), os.path.join(image_dst, img))
    shutil.move(os.path.join(label_src, lab), os.path.join(label_dst, lab))    


2413it [00:00, 24441.49it/s]
